<a href="https://colab.research.google.com/github/Anchaliya75/Brain-Computer-Interface/blob/main/TestingA8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader as dl
import matplotlib.pyplot as plt

In [26]:
from scipy.io import loadmat
matfile  ="/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A01T.mat"
matfile2 ="/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A02T.mat"
matfile3= "/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A03T.mat"
matfile4 = "/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A04T.mat"
matfile5 = "/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A05T.mat"
matfile6= "/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A06T.mat"
matfile7 = "/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A07T.mat"
matfile8 ="/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A08T.mat"
matfile9 = "/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A09T.mat"
matfilee1="/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A01E.mat"
matfilee2= "/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A02E.mat"
matfilee3="/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A03E.mat"
matfilee4="/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A04E.mat"
matfilee5="/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A05E.mat"
matfilee6="/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A06E.mat"
matfilee7="/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A07E.mat"
matfilee8="/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A08E.mat"
matfilee9="/content/drive/MyDrive/BCI Competition IV 2a dataset/BCI/A09T.mat"
data1 = loadmat(matfile)['data']
data2 = loadmat(matfile2)['data']
data3 = loadmat(matfile3)['data']
data4 = loadmat(matfile4)['data']
data5 = loadmat(matfile5)['data']
data6 = loadmat(matfile6)['data']
data7 = loadmat(matfile7)['data']
data8 = loadmat(matfile8)['data']
data9 = loadmat(matfile9)['data']
data1e= loadmat(matfilee1)['data']
data2e= loadmat(matfilee2)['data']
data3e= loadmat(matfilee3)['data']
data4e= loadmat(matfilee4)['data']
data5e= loadmat(matfilee5)['data']
data6e= loadmat(matfilee6)['data']
data7e= loadmat(matfilee7)['data']
data8e= loadmat(matfilee8)['data']
data9e= loadmat(matfilee9)['data']


#Removing data3e due to smaller data-interval

# all_data = [data1,data2,data3,data4,data5,data6,data7,data8,data9]
# all_data_test =[data1e,data2e,data3e,data4e,data5e,data6e,data7e,data8e,data9e]
all_data  = [data1,data2,data3,data4,data5,data6,data7,data8,data9,data1e,data2e,data3e,data5e,data6e,data8e,data9e]
all_data_test = [data7e]
# Doing Cross Validation on all

In [27]:
batch_size=288

In [28]:
def sampling(x):
  mini =1e9
  ma=0
  for  i in range(len(x)):
        mini =min(mini,x[i].shape[0])
        x_updated=[]
  for i  in range(len(x)):
        x_a = x[i][:mini-5]
        x_updated.append(x_a)
     
  return x_updated




In [29]:
def converting_mat_to_tensors(all_data,x,y):
  subjects  = len(all_data)
  for  a  in tqdm(range(subjects)):
      data = all_data[a]
      
      if(a==3):
        
        for i in range(1,7):
          samples = data[0,i][0,0][1]
          n =len(samples)
          prev= data[0,i][0,0][1][0][0]
          for j in range(1,n):
            take  = data[0,i][0,0][1][j][0]
            x.append(data[0,i][0,0][0][prev:take,0:22])
            y.append(data[0,i][0,0][2][j-1][0])
            prev = take
          x.append(data[0,i][0,0][0][prev:,0:22]) 
          y.append(data[0,i][0,0][2][j][0])

      else:
        
        for i in range(3,9):
          samples = data[0,i][0,0][1]
          n =len(samples)
          prev= data[0,i][0,0][1][0][0]
          for j in range(1,n):
            take  = data[0,i][0,0][1][j][0]
            x.append(data[0,i][0,0][0][prev:take,0:22])
            y.append(data[0,i][0,0][2][j-1][0])
            prev = take
          x.append(data[0,i][0,0][0][prev:,0:22]) 
          y.append(data[0,i][0,0][2][j][0])
  a=sampling(x)
  a=torch.tensor(a)
  dim1=a.shape[0]
  dim2=a.shape[1]
  dim3=a.shape[2]
  a=a.reshape(dim1,1,dim2,dim3)
  return a,torch.tensor(y)
  
  
  


In [30]:
def converting_mat_to_tensors_test(all_data,x,y):
  subjects  = len(all_data)
  for  a  in tqdm(range(subjects)):
      data = all_data[a]
      
      if(a==3):
        
        for i in range(3,9):
          samples = data[0,i][0,0][1]
          n =len(samples)
          prev= data[0,i][0,0][1][0][0]
          for j in range(1,n):
            take  = data[0,i][0,0][1][j][0]
            x.append(data[0,i][0,0][0][prev:take,0:22])
            y.append(data[0,i][0,0][2][j-1][0])
            prev = take
          x.append(data[0,i][0,0][0][prev:,0:22]) 
          y.append(data[0,i][0,0][2][j][0])

      else:
        
        for i in range(3,9):
          samples = data[0,i][0,0][1]
          n =len(samples)
          prev= data[0,i][0,0][1][0][0]
          for j in range(1,n):
            take  = data[0,i][0,0][1][j][0]
            x.append(data[0,i][0,0][0][prev:take,0:22])
            y.append(data[0,i][0,0][2][j-1][0])
            prev = take
          x.append(data[0,i][0,0][0][prev:,0:22]) 
          y.append(data[0,i][0,0][2][j][0])
  a=sampling(x)
  a=torch.tensor(a)
  dim1=a.shape[0]
  dim2=a.shape[1]
  dim3=a.shape[2]
  a=a.reshape(dim1,1,dim2,dim3)
  return a,torch.tensor(y)
  
  
  


In [31]:
def preperation_for_dataloader(x,y):
  array=[]
  for i in range(len(y)):
    array.append((x[i],y[i]))
  return array


In [32]:
x,y=converting_mat_to_tensors(all_data,[],[])


  0%|          | 0/16 [00:00<?, ?it/s]

In [33]:
x.shape

torch.Size([4608, 1, 1890, 22])

In [34]:
data_loader_prep = preperation_for_dataloader(x,y)


In [ ]:
# (a,b)=data_loader_prep[0]
# print(a.size())

In [35]:
train_batch = dl(data_loader_prep ,batch_size=288)

In [36]:
for idx,(a,b) in enumerate(train_batch):
  print(a.shape)
  print(b.shape)
  break

torch.Size([288, 1, 1890, 22])
torch.Size([288])


In [37]:
class BCI(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(in_channels=1 ,out_channels=3,kernel_size=3 ,padding =1)
    self.maxpool =nn.MaxPool2d(2)
    self.linear1 = nn.Linear(945*11*3,500)
    self.linear2=nn.Linear(500,125)
    self.linear3=nn.Linear(125,5)
  def forward(self,input_data):

    x= self.conv1(input_data)
    
    x= self.maxpool(x)
    
    x=x.reshape(288,-1)
    x= F.relu(self.linear1(x))
    x=F.relu(self.linear2(x))
    x=self.linear3(x)
    return x



In [38]:
brain = BCI()
lossfunction  =nn.CrossEntropyLoss()
optimizer = optim.Adam(brain.parameters(),lr =0.001)

In [50]:
brain

BCI(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=31185, out_features=500, bias=True)
  (linear2): Linear(in_features=500, out_features=125, bias=True)
  (linear3): Linear(in_features=125, out_features=5, bias=True)
)

In [39]:
loss_array=[]
epochs = 15
for  i in range(epochs):
  for idx,(x_train,y_train) in enumerate(train_batch):
    brain.zero_grad()
    prediction =brain.forward(x_train.float())
    loss=lossfunction(prediction,y_train)
    loss.backward()
    optimizer.step()
  print(loss)

    



tensor(3.6162, grad_fn=<NllLossBackward0>)
tensor(5.4144, grad_fn=<NllLossBackward0>)
tensor(1.1345, grad_fn=<NllLossBackward0>)
tensor(3.4866, grad_fn=<NllLossBackward0>)
tensor(2.6037, grad_fn=<NllLossBackward0>)
tensor(0.9854, grad_fn=<NllLossBackward0>)
tensor(0.8592, grad_fn=<NllLossBackward0>)
tensor(0.6181, grad_fn=<NllLossBackward0>)
tensor(0.5162, grad_fn=<NllLossBackward0>)
tensor(0.4637, grad_fn=<NllLossBackward0>)
tensor(0.3945, grad_fn=<NllLossBackward0>)
tensor(0.3575, grad_fn=<NllLossBackward0>)
tensor(0.3067, grad_fn=<NllLossBackward0>)
tensor(0.3005, grad_fn=<NllLossBackward0>)
tensor(0.4629, grad_fn=<NllLossBackward0>)


In [40]:
x_test,y_test = converting_mat_to_tensors_test(all_data_test,[],[])

  0%|          | 0/1 [00:00<?, ?it/s]

In [41]:
data_loader_prep_test = preperation_for_dataloader(x_test,y_test)


In [42]:
test_batch = dl(data_loader_prep_test ,batch_size=288)

In [43]:
correct_train =0
total_train =0
with torch.no_grad():
  for idx,(x_train,y_train) in enumerate(train_batch):
    prediction = brain.forward(x_train.float())
    
    for idx,predicted_probability in enumerate(prediction):
      if(torch.argmax(predicted_probability)==y_train[idx]):
        correct_train+=1
      total_train+=1


In [44]:
print(correct_train)

2968


In [45]:
print(total_train)

4608


In [47]:
print("Training accuracy is",correct_train/total_train)

Training accuracy is 0.6440972222222222


In [ ]:
def make_plot(y):
  n=len(y)
  y=np.array(y)

  x=[]
  for i in range(n):
    x.append(i+1)
  x=np.array(x)
  plt.plot(x,y,x_label="no_of_epochs",y_label="training_loss")
  


In [48]:
correct =0
total =0
with torch.no_grad():
  for idx,(x_test,y_test) in enumerate(test_batch):
    prediction = brain.forward(x_test.float())
    
    for idx,predicted_probability in enumerate(prediction):
      if(torch.argmax(predicted_probability)==y_test[idx]):
        correct+=1
      total+=1


In [49]:

print("Test accuracy is",correct/total)

Test accuracy is 0.4027777777777778
